In [1]:
from ctypes import *
from ctypes.wintypes import *
import numpy as np

In [2]:
PROCESS_ID = 8028

PLAYERS_START_ADDR = 0x04f26000
OFFSET = 0x20
START_ADDR = PLAYERS_START_ADDR + OFFSET

PLAYER_ADDR_STRLEN = 4

PROCESS_VM_READ = 0x0010

PLAYER_CLASS_ID = 0x0133f1a80133ec44

In [3]:
k32 = WinDLL('kernel32')
k32.OpenProcess.argtypes = DWORD,BOOL,DWORD
k32.OpenProcess.restype = HANDLE
k32.ReadProcessMemory.argtypes = HANDLE,LPVOID,LPVOID,c_size_t,POINTER(c_size_t)
k32.ReadProcessMemory.restype = BOOL

In [4]:
def reverse_array(array):
    length = len(array)
    left = 0
    right = length-1
    while left < right:
        temp = array[left]
        array[left] = array[right]
        array[right] = temp
        left = left + 1
        right = right - 1

        
def reverse_hex_string(st):
    length = len(st)
    st = list(st)
    left = 0
    right = length
    while left < right:
        temp = st[left:left+2]
        st[left:left+2] = st[right-2:right]
        st[right-2:right] = temp
        left = left + 2
        right = right - 2
    return "".join(st)

def attribute_100_20(int_value):
    return int_value
        
def decode_player(addr):
    
    process = k32.OpenProcess(PROCESS_VM_READ, False, PROCESS_ID)
    buf = create_string_buffer(PLAYER_ADDR_STRLEN)
    s = c_size_t()
    
    uid_addr = addr + 232
    
    if k32.ReadProcessMemory(process, uid_addr, buf, 4, byref(s)):
        reverse_array(buf)
        value = int(buf.raw.hex(),16)
        return value
    else:
        print("Access Denied!")
        return -1
    
def decode_player_class_id(addr):
    
    process = k32.OpenProcess(PROCESS_VM_READ, False, PROCESS_ID)
    buf = create_string_buffer(8)
    s = c_size_t()
    
    if k32.ReadProcessMemory(process, addr, buf, 8, byref(s)):
        reverse_array(buf)
        value = int(buf.raw.hex(),16)
        return value
    else:
        print("Access Denied!")
        return -1

In [5]:
process = k32.OpenProcess(PROCESS_VM_READ, False, PROCESS_ID)
buf = create_string_buffer(PLAYER_ADDR_STRLEN)
s = c_size_t()

addr = START_ADDR

number = 0

while True:
    if k32.ReadProcessMemory(process, addr, buf, PLAYER_ADDR_STRLEN, byref(s)):
        reverse_array(buf)
        value = int(buf.raw.hex(),16)
        if value == 0:
            break
        number = number + 1
        addr = addr + PLAYER_ADDR_STRLEN
    else:
        print("Access Denied!")
        break
        
print(number)    

27311


In [ ]:
process = k32.OpenProcess(PROCESS_VM_READ, False, PROCESS_ID)
buf = create_string_buffer(PLAYER_ADDR_STRLEN)
s = c_size_t()

addr = START_ADDR

while True:
    
    if k32.ReadProcessMemory(process, addr, buf, PLAYER_ADDR_STRLEN, byref(s)):
        reverse_array(buf)
        player_addr = int(buf.raw.hex(),16)
        if player_addr == 0:
            break
        uid = decode_player(player_addr)
        print(uid)
        addr = addr + PLAYER_ADDR_STRLEN
        if uid == -1:
            print("Returned -1, breaking")
            break
    else:
        print("Access Denied!")
        break  

In [ ]:
process = k32.OpenProcess(PROCESS_VM_READ, False, PROCESS_ID)
buf = create_string_buffer(PLAYER_ADDR_STRLEN)
s = c_size_t()

addr = START_ADDR

num_players = 0
num_other = 0

while True:
    
    if k32.ReadProcessMemory(process, addr, buf, PLAYER_ADDR_STRLEN, byref(s)):
        reverse_array(buf)
        player_addr = int(buf.raw.hex(),16)
        if player_addr == 0:
            break
        class_id = decode_player_class_id(player_addr)
        
        if class_id == PLAYER_CLASS_ID:
            num_players = num_players + 1
        elif class_id == -1:
            print("Returned -1, breaking")
            break
        else:
            print(addr)
            print(class_id)
            print("---------------")
            num_other = num_other + 1
        addr = addr + PLAYER_ADDR_STRLEN
    else:
        print("Access Denied!")
        break

print(num_players)
print(num_other)

In [6]:
PLAYER_CLASS_ID

86678521727151172

In [4]:
from util.player import player

In [5]:
p = player(PROCESS_ID)

In [7]:
process = k32.OpenProcess(PROCESS_VM_READ, False, PROCESS_ID)
buf = create_string_buffer(484)
s = c_size_t()

addr = 0x0c86682c

if k32.ReadProcessMemory(process, addr, buf, 484, byref(s)):
    print("success")
else:
    print("Access Denied!")

success


In [8]:
buf[1]

b'\xec'

In [9]:
buf[1].hex()

'ec'

In [10]:
int(buf[1].hex(),16)

236

In [6]:
p.populate_player_from_addr(0x0c86682c)

In [7]:
p

In [8]:
p.under_21_apps

2

In [9]:
p.uid

2000161838

In [10]:
p.eccentricity

8.0

In [11]:
p.total

521.0